<a href="https://colab.research.google.com/github/toche7/AIforAll/blob/main/word_embeddings_DATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Word embeddings in 2020. Review with code  examples

by [Rostyslav Neskorozhenyi](https://www.linkedin.com/in/slanj)

In this article we will study word embeddings - digital representation of words suitable for processing by machine learning algorithms.

Originally I created this article as a general overview and compilation of current approaches to word embeddings in 2020, which our [AI Labs](http://ai-labs.org/) team could use from time to time as a quick refresher. I hope that my article will be useful to a wider circle of data scientists and developers. Each word embedding method in the article has a (very) short description, links for further study, and code examples in Python. All code is packed as [Google Colab Notebook](https://colab.research.google.com/drive/1N7HELWImK9xCYheyozVP3C_McbiRo1nb). So let's begin.

According to Wikipedia, **Word embedding** is the collective name for a set of language modeling and feature learning techniques in natural language processing (NLP) where words or phrases from the vocabulary are mapped to vectors of real numbers.

## One-hot or CountVectorizing

The most basic method for transforming words into vectors is to count occurrence of each word in each document. Such approach is called countvectorizing or one-hot encoding.

The main principle of this method is to collect a set of documents (they can be words, sentences, paragraphs or even articles) and count the occurrence of every word in each document. Strictly speaking, the columns of the resulting matrix are words and the rows are documents.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# create CountVectorizer object
vectorizer = CountVectorizer()
corpus = [
          'Text of the very first new sentence with the first words in sentence.',
          'Text of the second sentence.',
          'Number three with lot of words words words.',
          'Short text, less words.',
]

In [ ]:
corpus

In [ ]:
# learn the vocabulary and store CountVectorizer sparse matrix in term_frequencies
term_frequencies = vectorizer.fit_transform(corpus)
vocab = vectorizer.get_feature_names_out()
vocab

In [ ]:
term_frequencies = term_frequencies.toarray() # convert sparse matrix to numpy array
term_frequencies

In [ ]:
import seaborn as sns
sns.heatmap(term_frequencies, annot=True, cbar = False, xticklabels = vocab);

In [ ]:
# Convert another document with countvectorizing
vectorizer.transform(['A new new sentence.']).toarray()

Another approach in countvectorizing is just to place 1 if the word is found in the document (no matter how often) and 0 if the word is not found in the document. In this case we get real 'one-hot' encoding.

In [ ]:
one_hot_vectorizer = CountVectorizer(binary=True)
one_hot = one_hot_vectorizer.fit_transform(corpus).toarray()
one_hot

In [ ]:
sns.heatmap(one_hot, annot=True, cbar = False, xticklabels = vocab)

## TF-IDF encoding

With a large corpus of documents some words like ‘a’, ‘the’, ‘is’, etc. occur very frequently but they don’t carry a lot of information. Using one-hot encoding approach we can decide that these words are important because they appear in many documents. One of the ways to solve this problem is stopwords filtering, but this solution is discrete and not flexible.

TF-IDF (term frequency - inverse document frequency) can deal with this problem better. TF-IDF lowers the weight of commonly used words and raises the weight of rare words that occur only in current document. TF-IDF formula looks like this:
<br><br>

$tfidf(term, document)= tf(term, document) \cdot idf(term)$

<br>
Where TF is calculated by dividing number of times the word occurs in the document by the total number of words in the document

$tf(term, document)= \frac{n_i}{\sum_{k=1}^W n_k}$

IDF (inverse document frequency), interpreted like inversed number of documents, in which the term we’re interested in occurs. N - number of documents, n(t) - number of documents with current word or term t.


$idf(term) = \log {\frac{N}{n_t}} $




In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import seaborn as sns

corpus = [
          'Time flies like an arrow.',
          'Fruit flies like a banana.'
]

vocab = ['an', 'arrow', 'banana', 'flies', 'fruit', 'like', 'time']

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(corpus).toarray()
tfidf

In [ ]:
sns.heatmap(tfidf, annot=True, cbar = False, xticklabels = vocab)

## Word2Vec and GloVe

The most commonly used models for word embeddings are [word2vec](https://github.com/dav/word2vec/) and [GloVe](https://nlp.stanford.edu/projects/glove/) which are both unsupervised approaches based on the distributional hypothesis (words that occur in the same contexts tend to have similar meanings).

Word2Vec word embeddings are vector representations of words,
that are typically learnt by an unsupervised model when fed
with large amounts of text as input (e.g. Wikipedia, science, news, articles etc.). These representation of words capture semantic similarity between words among other properties. Word2Vec word embeddings are learnt in a such way, that [distance](https://en.wikipedia.org/wiki/Euclidean_distance) between vectors for words with close meanings ("king" and "queen" for example) are closer than distance for words with complety different meanings ("king" and "carpet" for example).

![Замещающий текст](https://developers.google.com/machine-learning/crash-course/images/linear-relationships.svg)
Image from [developers.google.com](https://developers.google.com/machine-learning/crash-course/embeddings/translating-to-a-lower-dimensional-space)

Word2Vec vectors even allow some mathematic operations on vectors. For example, in this operation we are using word2vec vectors for each word:

**king - man + woman = queen**

Another word embedding method is **Glove** (“Global Vectors”). It is based on matrix factorization techniques on the word-context matrix. It first constructs a large matrix of (words x context) co-occurrence information, i.e. for each “word” (the rows), you count how frequently we see this word in some “context” (the columns) in a large corpus. Then this matrix is factorized to a lower-dimensional (word x features) matrix, where each row now stores a vector representation for each word. In general, this is done by minimizing a “reconstruction loss”. This loss tries to find the lower-dimensional representations which can explain most of the variance in the high-dimensional data.

In [ ]:
import spacy

In [ ]:
# Try Glove word embeddings with Spacy
!python3 -m spacy download en_core_web_lg

In [ ]:
import spacy
# Load the spacy model that you have installed
import en_core_web_lg
nlp = en_core_web_lg.load()
# process a sentence using the model
doc = nlp("man king stands on the carpet and sees woman queen")
# Get the vector for 'king':
doc[1].vector[:]

In [ ]:
for i in range(len(doc)):
  print(doc[i])

Find similarity between King and Queen (higher value is better).

In [ ]:
for (i, dToken) in enumerate(doc):
  print(i,dToken)

In [ ]:
doc[1].similarity(doc[5])

Find similarity between King and carpet

In [ ]:
doc[1].similarity(doc[5])

Check if king - man + woman = queen. We will multiply vectors for 'man' and 'woman' by two, because subtracting the vector for 'man' and adding the vector for 'woman' will do little to the original vector for “king”, likely because those “man” and “woman” are related themselves.

In [ ]:
v =  doc[1].vector - (doc[0].vector*3) + (doc[8].vector*3)

from scipy.spatial import distance
import numpy as np

# Format the vocabulary for use in the distance function
vectors = [token.vector for token in doc]
vectors = np.array(vectors)

# Find the closest word below
closest_index = distance.cdist(np.expand_dims(v, axis = 0), vectors, metric = 'cosine').argmin()
output_word = doc[closest_index].text

In [ ]:
output_word

## References

- [BERT Word Embeddings Tutorial](https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/)
- [The Illustrated Transformer](http://jalammar.github.io/illustrated-transformer/)
- [The Illustrated GPT-2 (Visualizing Transformer Language Models)](http://jalammar.github.io/illustrated-gpt2/)
- [FROM Pre-trained Word Embeddings TO Pre-trained Language Models — Focus on BERT](https://towardsdatascience.com/from-pre-trained-word-embeddings-to-pre-trained-language-models-focus-on-bert-343815627598)
- [ Make your own Rick Sanchez (bot) with Transformers and DialoGPT fine-tuning](https://towardsdatascience.com/make-your-own-rick-sanchez-bot-with-transformers-and-dialogpt-fine-tuning-f85e6d1f4e30)
- [Playing with word vectors](https://medium.com/swlh/playing-with-word-vectors-308ab2faa519)
- [Intuitive Guide to Understanding GloVe Embeddings](https://towardsdatascience.com/light-on-math-ml-intuitive-guide-to-understanding-glove-embeddings-b13b4f19c010)
- [Word Embeddings in Python with Spacy and Gensim](https://www.shanelynn.ie/word-embeddings-in-python-with-spacy-and-gensim/)
- [Brief review of word embedding families (2019) ](https://medium.com/analytics-vidhya/brief-review-of-word-embedding-families-2019-b2bbc601bbfe)
- [Word embeddings: exploration, explanation, and exploitation (with code in Python)](https://towardsdatascience.com/word-embeddings-exploration-explanation-and-exploitation-with-code-in-python-5dac99d5d795)